In [1]:
import os
import json
import tqdm
from collections import Counter
import numpy as np
from sys import getsizeof

# max length

In [14]:
with open('dict_targets_Reviews.json', 'r', encoding='UTF-8') as f:
    target_dict=json.load(f)

In [103]:
with open('dict_inputs_Reviews.json', 'r', encoding='UTF-8') as f:
    inputs_dict=json.load(f)

In [16]:
cnt = 0
cnt_list = []
target_cnt_list = []
input_cnt_list = []
key_cnt = ''
for key in target_dict.keys():
    target_cnt_list.append(len(target_dict[key]))
    input_cnt_list.append(len(inputs_dict[key]))
    if (len(target_dict[key]) > len(inputs_dict[key]) and len(target_dict[key]) > cnt):
        cnt = len(target_dict[key])
        cnt_list.append(len(target_dict[key]))
        key_cnt = key
    elif (len(target_dict[key]) < len(inputs_dict[key]) and len(inputs_dict[key]) > cnt):
        cnt = len(inputs_dict[key])
        cnt_list.append(len(inputs_dict[key]))
        key_cnt = key

In [17]:
input_cnt_list.sort()

In [18]:
input_cnt_list.index(400)/len(input_cnt_list)

0.9824498132565835

In [19]:
max_length = 400

In [20]:
with open('word_unk_dic.json', 'r', encoding='UTF-8') as f:
    word_unk_dic=json.load(f)

word_list = word_unk_dic['WORD']

unk_list = word_unk_dic['UNK']

# Pad

In [21]:
_PAD_ = "_PAD_"  # 빈칸 채우는 심볼
_STA_ = "_STA_"  # 디코드 입력 시퀀스의 시작 심볼
_EOS_ = "_EOS_"  # 디코드 입출력 시퀀스의 종료 심볼
_UNK_ = "_UNK_"  # 사전에 없는 단어를 나타내는 심볼

_PAD_ID_ = 0
_STA_ID_ = 1
_EOS_ID_ = 2
_UNK_ID_ = 3
_PRE_DEFINED_ = [_PAD_ID_, _STA_ID_, _EOS_ID_, _UNK_ID_]

_PRE_DEFINED_dict = {_PAD_:_PAD_ID_, _STA_:_STA_ID_, _EOS_:_EOS_ID_, _UNK_:_UNK_ID_}

In [22]:
def padding(seq, max_len, start=None, eos=None):
    if start:
        padded_seq = [_STA_] + seq
    elif eos:
        padded_seq = seq + [_EOS_]
    else:
        padded_seq = seq

    if len(padded_seq) < max_len:
        return padded_seq + ([_PAD_] * (max_len - len(padded_seq)))
    else:
        return padded_seq

In [137]:
def unk_make(data):
    
    for index, word in enumerate(data):

        if word in word_list:
            pass
        else:
            data[index] = _UNK_
            pass
            
    return data

In [138]:
def data_padding(data,max_len = 0, start=None, eos=None):
    pos_padding_dict = {}
    for title in tqdm.tqdm(list(data.keys())):
        if (len(data[title])>max_len):
            data[title] = data[title][:max_len]
        sentence = unk_make(data[title])
        pos_padding_dict[title] = padding(sentence,max_len,start,eos)
        
    return pos_padding_dict

In [139]:
targets = []
for key in target_dict.keys():
    targets.append(target_dict[key])

In [140]:
targets_pad = data_padding(target_dict,400,None,True)

100%|████████████████████████████████████████████████████████████████████████| 568427/568427 [01:10<00:00, 8093.54it/s]


In [182]:
with open('targets_pad.txt', 'w') as f:
    json.dump(targets_pad, f)

KeyboardInterrupt: 

In [141]:
inputs = []
for key in inputs_dict.keys():
    inputs.append(inputs_dict[key])

In [142]:
inputs_pad = data_padding(inputs_dict,400,True,None)

100%|█████████████████████████████████████████████████████████████████████████| 568427/568427 [16:08<00:00, 587.14it/s]


In [ ]:
with open('inputs_pad.txt', 'w') as f:
    json.dump(inputs_pad, f)

In [145]:
del target_dict
del inputs_dict

In [146]:
import gc
collected = gc.collect()
print (collected)

0


In [147]:
import gensim
from gensim.models.word2vec import Word2Vec
import sys
import multiprocessing

C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [148]:
cores = multiprocessing.cpu_count()

In [160]:
input_word2vec = []
for key in list(inputs_pad.keys()):
    input_word2vec.append(inputs_pad[key])
    input_word2vec.append(targets_pad[key])

In [163]:
%%time
model = Word2Vec(input_word2vec, size=300, workers=cores,sg=1 ,min_count=0)

Wall time: 9min 38s


In [168]:
%%time
# Train document vectors!
for epoch in tqdm.tqdm(range(10)):
    model.train(input_word2vec,total_examples=model.corpus_count,epochs=1,compute_loss=True)
    print(model.get_latest_training_loss())

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

39820744.0


 10%|████████▏                                                                         | 1/10 [01:58<17:50, 118.92s/it]

39897992.0


 20%|████████████████▍                                                                 | 2/10 [03:58<15:54, 119.32s/it]

...

40058380.0


 90%|█████████████████████████████████████████████████████████████████████████▊        | 9/10 [17:30<01:56, 116.77s/it]

40003352.0


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [19:46<00:00, 118.66s/it]


Wall time: 19min 46s


In [180]:
model.most_similar('large')

C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('small', 0.746677577495575),
 ('larger', 0.6277764439582825),
 ('smaller', 0.5631756782531738),
 ('big', 0.5600061416625977),
 ('sized', 0.5478004217147827),
 ('clamp-top', 0.4951903820037842),
 ('huge', 0.4892905354499817),
 ('giant', 0.4706695079803467),
 ('mug-full', 0.4593583345413208),
 ('generous', 0.45926225185394287)]

In [181]:
model.save("Word2vec_data.model")